## 데이터 및 라이브러리준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title 필요 라이브러리 import
import os
import re
import cv2
import heapq
import torch
import json
import glob
import shutil
from pathlib import Path
from itertools import cycle
import random
import albumentations as A
import numpy as np
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.patches as patches
from collections import Counter
from collections import defaultdict
from google.colab import files
import torchvision.transforms.functional as F2
from IPython.display import Image as IPImage, display
from shutil import copyfile
from PIL import Image, ImageDraw

In [ ]:
# @title matplotlib 한글 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

In [ ]:
# @title zip 파일 압축해제
!cp /content/drive/MyDrive/codeit/project/pill_data_v.zip /content/
!unzip -q /content/pill_data_v.zip -d /content/ai03-level1-project

In [ ]:
# @title 작업 경로 지정
path = "/content/ai03-level1-project"

os.chdir(path)

In [ ]:
# @title 디렉터리 경로지정 및 생성
root_dir = "/content/ai03-level1-project"
source_img_dir = "/content/ai03-level1-project/train_images"
source_label_dir = "/content/ai03-level1-project/train_annotations"
test_img_dir = "/content/ai03-level1-project/test_images"
merged_json_dir = "/content/ai03-level1-project/merge_ann"
yolo_label_dir = "/content/ai03-level1-project/labels"
yolo_image_dir = "/content/ai03-level1-project/images"
yolo_train_img_dir = "/content/ai03-level1-project/images/train_images"
yolo_val_img_dir = "/content/ai03-level1-project/images/val_images"
yolo_train_label_dir = "/content/ai03-level1-project/labels/train_images"
yolo_val_label_dir = "/content/ai03-level1-project/labels/val_images"
yaml_path = "/content/ai03-level1-project/data.yaml"
merged_json_path = os.path.join(merged_json_dir, "merged_annotations.json")
merged_filtered_json_path = os.path.join(merged_json_dir, "merged_filtered_annotations.json")

os.makedirs(merged_json_dir, exist_ok=True)
os.makedirs(yolo_label_dir, exist_ok=True)
os.makedirs(yolo_image_dir, exist_ok=True)
os.makedirs(merged_json_dir, exist_ok=True)
os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_val_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_val_label_dir, exist_ok=True)

In [ ]:
# @title 모델 저장경로 지정
drive_path = "/content/drive/MyDrive/codeit/project/model/transtest"
project_name = "tran_test"
project_path = os.path.join(drive_path, project_name)
# os.makedirs(project_path)

## 필요 메서드 정의

In [ ]:
# @title 어노테이션 병합
def merge_coco_jsons(root_json_dir, merged_json_path):
    """
    COCO 형식의 JSON 어노테이션 파일들을 병합하여 하나의 JSON으로 저장합니다.

    Args:
        root_json_dir (str): JSON 파일들이 위치한 최상위 디렉토리 경로. 하위 디렉토리까지 재귀적으로 탐색합니다.
        merged_json_path (str): 병합된 JSON 파일을 저장할 경로.

    Returns:
        None. 병합된 결과는 merged_json_path에 저장됩니다.
    """
    os.makedirs(os.path.dirname(merged_json_path), exist_ok=True)

    json_files = glob.glob(os.path.join(root_json_dir, '**', '*.json'), recursive=True)
    print(f"[DEBUG] 찾은 JSON 파일 개수: {len(json_files)}")
    print(f"[DEBUG] 예시 JSON 파일들: {json_files[:5]}")

    merged = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    annotation_id = 1
    image_id_set = set()
    category_set = set()

    for json_file in json_files:
        with open(json_file, 'r') as f:
            data = json.load(f)

        for img in data.get("images", []):
            if img["id"] not in image_id_set:
                merged["images"].append(img)
                image_id_set.add(img["id"])

        for ann in data.get("annotations", []):
            ann["id"] = annotation_id
            annotation_id += 1
            merged["annotations"].append(ann)

        for cat in data.get("categories", []):
            if cat["id"] not in category_set:
                merged["categories"].append(cat)
                category_set.add(cat["id"])

    with open(merged_json_path, 'w') as f:
        json.dump(merged, f, indent=2)

    print(f"[DEBUG] 병합된 JSON 저장 완료: {merged_json_path}")

# 어노테이션 병합
merge_coco_jsons(source_label_dir, merged_json_path)

In [ ]:
# @title 이미지파일 yolo 이미지 디렉터리로 복사
def copy_images(file_names, source_img_dir, target_img_dir):
    """
    지정된 이미지 파일들을 원본 디렉토리에서 대상 디렉토리로 복사합니다.

    Args:
        file_names (list): 복사할 이미지 파일 이름 리스트.
        source_img_dir (str): 원본 이미지들이 있는 디렉토리 경로.
        target_img_dir (str): 이미지들을 복사할 대상 디렉토리 경로.

    Returns:
        None. 파일이 target_img_dir로 복사됩니다.
    """
    os.makedirs(target_img_dir, exist_ok=True)
    copied_count = 0
    for fname in file_names:
        source_path = os.path.join(source_img_dir, fname)
        target_path = os.path.join(target_img_dir, fname)
        if os.path.exists(source_path):
            shutil.copy2(source_path, target_path)
            copied_count += 1
        else:
            print(f"[WARNING] 원본 이미지 없음: {source_path}")
    print(f"이미지 복사 완료: {copied_count}개")

In [ ]:
# @title 학습/검증 데이터 분리
def split_train_val(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, val_ratio=0.15, seed=42):
    """
    이미지와 라벨 데이터를 학습용(train)과 검증용(val)으로 분리하여 각각의 디렉토리로 이동시킵니다.

    Args:
        image_dir (str): 원본 이미지 파일들이 있는 디렉토리 경로.
        label_dir (str): 원본 라벨(txt) 파일들이 있는 디렉토리 경로.
        train_img_dir (str): 학습용 이미지 저장 디렉토리.
        val_img_dir (str): 검증용 이미지 저장 디렉토리.
        train_label_dir (str): 학습용 라벨 저장 디렉토리.
        val_label_dir (str): 검증용 라벨 저장 디렉토리.
        val_ratio (float, optional): 전체 데이터 중 검증 데이터 비율. 기본값은 0.15.
        seed (int, optional): 데이터 분할을 위한 랜덤 시드. 기본값은 42.

    Returns:
        None. 이미지와 라벨 파일이 지정된 디렉토리로 이동됩니다.
    """
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort()
    random.seed(seed)
    random.shuffle(image_files)

    val_count = int(len(image_files) * val_ratio)
    val_images = set(image_files[:val_count])
    train_images = set(image_files[val_count:])

    for img_file in image_files:
        label_file = os.path.splitext(img_file)[0] + ".txt"

        src_img_path = os.path.join(image_dir, img_file)
        src_label_path = os.path.join(label_dir, label_file)

        if img_file in val_images:
            dst_img_path = os.path.join(val_img_dir, img_file)
            dst_label_path = os.path.join(val_label_dir, label_file)
        else:
            dst_img_path = os.path.join(train_img_dir, img_file)
            dst_label_path = os.path.join(train_label_dir, label_file)

        # 복사 → 이동
        if os.path.exists(src_img_path):
            shutil.move(src_img_path, dst_img_path)

        if os.path.exists(src_label_path):
            shutil.move(src_label_path, dst_label_path)

In [ ]:
# @title 학습/검증 데이터 분리
def split_by_class(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, val_ratio=0.15, seed=42):
    """
    이미지와 YOLO 라벨 데이터를 클래스 기준으로 학습/검증 세트로 분리하여
    클래스별로 디렉토리에 정리합니다.

    Args:
        image_dir (str): 원본 이미지 디렉토리 경로.
        label_dir (str): 원본 라벨(txt) 디렉토리 경로.
        train_img_dir (str): 학습 이미지 저장 디렉토리.
        val_img_dir (str): 검증 이미지 저장 디렉토리.
        train_label_dir (str): 학습 라벨 저장 디렉토리.
        val_label_dir (str): 검증 라벨 저장 디렉토리.
        val_ratio (float, optional): 검증 데이터 비율 (기본값: 0.15).
        seed (int, optional): 랜덤 시드 (기본값: 42).

    Returns:
        None. 클래스별로 분리된 이미지 및 라벨 파일이 각 디렉토리에 이동됩니다.
    """
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)

    # 이미지 파일 목록
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort()
    random.seed(seed)
    random.shuffle(image_files)

    # 각 클래스별로 이미지 파일을 나눌 준비
    class_train_images = defaultdict(list)
    class_val_images = defaultdict(list)

    # 이미지 분리 비율에 맞춰서 학습/검증 데이터 분할
    val_count = int(len(image_files) * val_ratio)
    val_images = set(image_files[:val_count])
    train_images = set(image_files[val_count:])

    # 각 이미지에 해당하는 라벨 파일 분리
    for img_file in image_files:
        label_file = os.path.splitext(img_file)[0] + ".txt"

        # 라벨 파일을 읽어서 클래스 정보를 추출
        src_img_path = os.path.join(image_dir, img_file)
        src_label_path = os.path.join(label_dir, label_file)

        # 라벨이 존재하면 해당 클래스들을 추출
        if os.path.exists(src_label_path):
            with open(src_label_path, 'r') as f:
                classes = [line.strip().split()[0] for line in f.readlines()]

            # 클래스별로 이미지 분류
            for cls in classes:
                if img_file in val_images:
                    class_val_images[cls].append(img_file)
                else:
                    class_train_images[cls].append(img_file)

    # 이미지 파일을 클래스별로 복사
    def move_files(class_images, source_img_dir, source_label_dir, target_img_dir, target_label_dir):
        """
        클래스별로 이미지와 라벨 파일을 지정된 디렉토리로 이동합니다.

        Args:
            class_images (dict): 클래스별 이미지 파일 리스트 딕셔너리.
            source_img_dir (str): 원본 이미지 디렉토리 경로.
            source_label_dir (str): 원본 라벨 디렉토리 경로.
            target_img_dir (str): 타겟 이미지 디렉토리 경로.
            target_label_dir (str): 타겟 라벨 디렉토리 경로.

        Returns:
            None.
        """
        for cls, img_files in class_images.items():
            cls_img_dir = os.path.join(target_img_dir, cls)
            cls_label_dir = os.path.join(target_label_dir, cls)
            os.makedirs(cls_img_dir, exist_ok=True)
            os.makedirs(cls_label_dir, exist_ok=True)

            for img_file in img_files:
                label_file = os.path.splitext(img_file)[0] + ".txt"

                # 이미지 파일 이동
                src_img_path = os.path.join(source_img_dir, img_file)
                dst_img_path = os.path.join(cls_img_dir, img_file)
                shutil.move(src_img_path, dst_img_path)

                # 라벨 파일 이동
                src_label_path = os.path.join(source_label_dir, label_file)
                dst_label_path = os.path.join(cls_label_dir, label_file)
                shutil.move(src_label_path, dst_label_path)

    # 학습 데이터와 검증 데이터에 대해 클래스별로 파일 이동
    move_files(class_train_images, image_dir, label_dir, train_img_dir, train_label_dir)
    move_files(class_val_images, image_dir, label_dir, val_img_dir, val_label_dir)

    print("클래스별로 이미지와 라벨 파일이 분리되었습니다.")

In [ ]:
# @title 이미지파일명 리스트 추출
def get_image_files_from_coco_json(json_path):
    """
    COCO 형식의 JSON 파일에서 이미지 파일명 리스트를 추출합니다.

    Args:
        json_path (str): COCO JSON 어노테이션 파일 경로.

    Returns:
        list: 이미지 파일 이름들이 담긴 리스트.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    image_files = [img['file_name'] for img in data.get('images', [])]
    return image_files

In [ ]:
# @title 카테고리 id, name 매핑
def get_master_categories(json_paths):
    """
    COCO JSON 파일들에서 category id와 name 정보를 추출하여 매핑 딕셔너리를 생성합니다.

    Args:
        json_paths (list): COCO 어노테이션 JSON 파일 경로들의 리스트.

    Returns:
        tuple:
            - cat_id_to_index (dict): 정렬된 category ID를 기준으로 0부터 시작하는 인덱스를 매핑한 딕셔너리.
            - cat_id_to_name (dict): category ID를 category 이름으로 매핑한 딕셔너리.
    """
    cat_id_to_name = {}
    for path in json_paths:
        with open(path, 'r') as f:
            data = json.load(f)
        for cat in data['categories']:
            cat_id_to_name[cat['id']] = cat['name']

    sorted_cat_ids = sorted(cat_id_to_name.keys())
    cat_id_to_index = {cat_id: idx for idx, cat_id in enumerate(sorted_cat_ids)}
    return cat_id_to_index, cat_id_to_name

In [ ]:
# @title coco 형식의 어노테이션 정보 yolo 형식으로 변환
def convert_coco_to_yolo(annotation_file, source_img_dir, target_yolo_dir, master_cat_id_to_idx):
    """
    COCO 형식의 어노테이션 파일을 YOLO 형식으로 변환하여 저장합니다.

    Args:
        annotation_file (str): COCO JSON 어노테이션 파일 경로.
        source_img_dir (str): 이미지 파일들이 저장된 디렉토리 경로.
        target_yolo_dir (str): 변환된 YOLO 라벨(txt) 파일들을 저장할 디렉토리 경로.
        master_cat_id_to_idx (dict): COCO category_id를 YOLO 클래스 인덱스로 매핑한 딕셔너리.

    Returns:
        None. YOLO 라벨 파일들이 target_yolo_dir에 저장됩니다.
    """
    os.makedirs(target_yolo_dir, exist_ok=True)
    with open(annotation_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    img_id_to_file = {img['id']: img['file_name'] for img in data['images']}
    anns_per_img = defaultdict(list)
    for ann in data['annotations']:
        anns_per_img[ann['image_id']].append(ann)

    for img_id, anns in anns_per_img.items():
        img_file = img_id_to_file[img_id]
        label_path = os.path.join(target_yolo_dir, img_file.rsplit('.', 1)[0] + ".txt")
        image_path = os.path.join(source_img_dir, img_file)

        if not os.path.exists(image_path):
            print(f"[경고] 이미지 없음: {image_path}")
            continue

        img = Image.open(image_path)
        width, height = img.size

        with open(label_path, 'w', encoding='utf-8') as f:
            for ann in anns:
                x, y, w, h = ann['bbox']
                xc = (x + w / 2) / width
                yc = (y + h / 2) / height
                wn = w / width
                hn = h / height
                cls_idx = master_cat_id_to_idx.get(ann['category_id'], None)
                if cls_idx is None:
                    print(f"[경고] category_id {ann['category_id']}가 master categories에 없음")
                    continue
                f.write(f"{cls_idx} {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}\n")

In [ ]:
# @title 완전라벨 데이터(약물수 == annotation수) 남기고 image/annotation파일 삭제
def count_drugs_from_filename(filename):
    """
    파일 이름에서 약물 개수를 추출합니다.
    예: "K-003351-020238-033880_..." → 약물 개수: 3

    Args:
        filename (str): 이미지 파일 이름.

    Returns:
        int: 추정된 약물 개수 (하이픈으로 구분된 prefix 기준).
    """
    prefix = filename.split('_')[0] # prefix 추출

    # 'K-' 접두어 제거
    if prefix.startswith("K-"):
        prefix = prefix[2:]

    return len(prefix.split('-'))

def clean_invalid_yolo_files_from_dir(merged_json_path, image_dir, label_dir):
    """
    이미지 파일에서 약물 개수(파일명 기반)와 COCO 어노테이션 내 실제 annotation 수(category 수)를 비교하여,
    불완전한 이미지 및 라벨 파일을 삭제합니다.

    Args:
        merged_json_path (str): 병합된 COCO 어노테이션 JSON 파일 경로.
        image_dir (str): 이미지 파일들이 저장된 디렉토리.
        label_dir (str): 라벨(.json) 파일들이 저장된 디렉토리 (하위 폴더 포함).

    Returns:
        None. 조건에 맞지 않는 이미지 및 해당 라벨 파일이 삭제됩니다.
    """

    with open(merged_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print("📂 JSON 로드 완료")

    file_to_id = {img["file_name"]: img["id"] for img in data["images"]}

    image_id_to_cat_ids = defaultdict(set)
    for ann in data["annotations"]:
        image_id_to_cat_ids[ann["image_id"]].add(ann["category_id"])

    removed = 0
    print(f"🔍 검사 중: {image_dir} / {label_dir}")

    for fname in os.listdir(image_dir):
        if not fname.endswith(".png"):
            continue

        base_name = os.path.splitext(fname)[0]
        full_name = fname
        drug_count = count_drugs_from_filename(fname)

        image_id = file_to_id.get(full_name)
        if image_id is None:
            print(f"⚠️ JSON에 없음: {full_name}")
            continue

        actual_category_count = len(image_id_to_cat_ids[image_id])

        if actual_category_count < drug_count:
            # 이미지 삭제
            img_path = os.path.join(image_dir, full_name)
            if os.path.exists(img_path):
                os.remove(img_path)
                print(f"🗑️ 이미지 삭제: {img_path}")

            # label_dir 아래 모든 하위 폴더에서 .json 삭제
            json_pattern = os.path.join(label_dir, "**", base_name + ".json")
            json_paths = glob.glob(json_pattern, recursive=True)

            for json_path in json_paths:
                if os.path.exists(json_path):
                    os.remove(json_path)
                    print(f"🗑️ JSON 삭제: {json_path}")

            removed += 1

    print(f"\n✅ 총 삭제된 항목 수: {removed}개\n")

In [ ]:
# @title 클래스별 이미지 수 계산
def count_classes_in_yolo_txt(labels_dir):
    """
    YOLO 형식의 라벨 파일을 읽어 클래스별 이미지 수 계산

    Args:
        labels_dir (str): 라벨 파일들이 있는 디렉토리 경로.

    Returns:
        class_count (dict): 클래스 ID별로 이미지 수를 카운트한 딕셔너리.
    """
    class_count = defaultdict(int)  # 각 클래스별 카운트를 저장하는 딕셔너리

    for root, _, files in os.walk(labels_dir):
        for file in files:
            if file.endswith(".txt"):
                label_path = os.path.join(root, file)

                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        parts = line.strip().split()  # YOLO 포맷에서 한 줄을 공백으로 나누기
                        if len(parts) >= 1:
                            try:
                                class_id = int(float(parts[0]))
                            except ValueError:
                                print(f"[경고] 클래스 ID 변환 실패: {parts[0]} in {file}")
                                continue
                            class_count[class_id] += 1  # 해당 클래스 ID에 대한 카운트 증가

    return class_count

In [ ]:
# @title 이미지, 라벨 개수 확인
def check_images_labels(img_dir, labels_dir):
    """
    이미지 디렉토리와 라벨 디렉토리 내 파일 수를 비교하고,
    이미지에 대응하는 라벨 파일 존재 여부 및 라벨에 대응하는 이미지 존재 여부를 검사합니다.

    Args:
        img_dir (str): 이미지 파일들이 있는 디렉토리 경로.
        labels_dir (str): YOLO 형식 라벨(.txt) 파일들이 있는 디렉토리 경로.

    Returns:
        bool: 이미지와 라벨이 완벽히 매칭되면 True, 그렇지 않으면 False.
    """
    if not os.path.exists(labels_dir):
        print(f"❌ 라벨 폴더가 없습니다: {labels_dir}")
        return False

    # 이미지 파일 리스트 (확장자 무시하고 이름만 추출)
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith('.png')]
    img_names = set(os.path.splitext(f)[0] for f in img_files)

    # 라벨 파일 리스트 (확장자 무시하고 이름만 추출)
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    label_names = set(os.path.splitext(f)[0] for f in label_files)

    print(f"이미지 파일 개수: {len(img_files)}")
    print(f"라벨 파일 개수: {len(label_files)}")

    # 이미지에 매핑되는 라벨 없는 파일들
    missing_labels = img_names - label_names
    # 라벨에 매핑되는 이미지 없는 파일들 (라벨만 있는 경우)
    missing_images = label_names - img_names

    if missing_labels:
        print(f"❌ 라벨이 없는 이미지 파일 ({len(missing_labels)}개):")
        for name in sorted(missing_labels):
            print(f"  - {name}")
    else:
        print("✅ 모든 이미지에 대응하는 라벨이 존재합니다.")

    if missing_images:
        print(f"⚠️ 라벨은 있으나 이미지가 없는 파일 ({len(missing_images)}개):")
        for name in sorted(missing_images):
            print(f"  - {name}")
    else:
        print("✅ 모든 라벨 파일에 대응하는 이미지가 존재합니다.")

    return len(missing_labels) == 0 and len(missing_images) == 0

In [ ]:
# @title 이미지 증강
def yolo_to_xyxy(bbox, img_w, img_h):
    """
    YOLO 형식의 bbox(x_center, y_center, width, height, normalized)를
    좌표 (x1, y1, x2, y2, 절대 픽셀 단위)로 변환합니다.

    Args:
        bbox (list): [x_center, y_center, width, height] (0~1 사이 정규화된 값)
        img_w (int): 이미지 너비 (픽셀)
        img_h (int): 이미지 높이 (픽셀)

    Returns:
        list: [x1, y1, x2, y2] 절대 좌표
    """
    x, y, w, h = bbox
    x1 = (x - w / 2) * img_w
    y1 = (y - h / 2) * img_h
    x2 = (x + w / 2) * img_w
    y2 = (y + h / 2) * img_h
    return [x1, y1, x2, y2]

def xyxy_to_yolo(bbox, img_w, img_h):
    """
    bbox (x1, y1, x2, y2, 절대 픽셀 단위)를 YOLO 형식의
    (x_center, y_center, width, height, normalized)로 변환합니다.

    Args:
        bbox (list): [x1, y1, x2, y2] 절대 좌표
        img_w (int): 이미지 너비 (픽셀)
        img_h (int): 이미지 높이 (픽셀)

    Returns:
        list: [x_center, y_center, width, height] (0~1 사이 정규화된 값)
    """
    x1, y1, x2, y2 = bbox
    x = ((x1 + x2) / 2) / img_w
    y = ((y1 + y2) / 2) / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    return [x, y, w, h]

# 증강 기법 리스트 (Albumentations 라이브러리 사용)
AUGMENTATIONS = [
    A.Compose([A.HorizontalFlip(p=1.0)]),
    A.Compose([A.VerticalFlip(p=1.0)]),
    A.Compose([A.RandomBrightnessContrast(p=1.0)]),
    A.Compose([A.Rotate(limit=25, p=1.0)]),
    A.Compose([A.GaussianBlur(p=1.0)]),
    A.Compose([A.ColorJitter(p=1.0)]),
    A.Compose([A.RandomGamma(p=1.0)])
]

def copy_few_shot_images(
    yolo_img_dir, yolo_label_dir,
    cat_id_to_name,
    max_classes_threshold=75,
    top_n=10,
    padding_ratio=0
):
    """
    희소 클래스(출현 빈도가 적은 클래스)에 대해 이미지 증강을 수행합니다.
    - 상위 top_n 클래스는 증강 제외(또는 크롭 증강 제외).
    - 희소 클래스별 이미지 수가 max_classes_threshold를 넘도록 증강 진행.
    - 기존 이미지를 기반으로 크롭 및 여러 변환을 적용해 증강 이미지 생성.

    Args:
        yolo_img_dir (str): YOLO 이미지 파일 경로
        yolo_label_dir (str): YOLO 라벨(.txt) 파일 경로
        cat_id_to_name (dict): 클래스 ID → 클래스 이름 매핑
        max_classes_threshold (int): 각 클래스별 최대 증강 이미지 개수
        top_n (int): 증강에서 제외할 상위 빈도 클래스 개수
        padding_ratio (float): 크롭 영역 주변 padding 비율 (0이면 패딩 없음)

    Returns:
        None. 증강된 이미지 및 라벨 파일을 지정 경로에 저장합니다.
    """
    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)

    class_img_map = defaultdict(set)
    img_class_map = defaultdict(set)

    # 클래스별 이미지, 이미지별 클래스 초기화
    for label_fname in os.listdir(yolo_label_dir):
        if not label_fname.endswith(".txt"):
            continue
        base_name = os.path.splitext(label_fname)[0]
        label_path = os.path.join(yolo_label_dir, label_fname)
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls_id = int(parts[0])
                class_img_map[cls_id].add(base_name)
                img_class_map[base_name].add(cls_id)

    class_counts = {cid: len(imgs) for cid, imgs in class_img_map.items()}
    top_n_classes = set(heapq.nlargest(top_n, class_counts, key=class_counts.get))

    few_classes = {
        cid: list(imgs)
        for cid, imgs in class_img_map.items()
        if (len(imgs) <= max_classes_threshold and cid not in top_n_classes)
    }

    sorted_few_class_ids = sorted(few_classes.keys(), key=lambda x: class_counts[x])

    def crop_and_save(image, bbox, cls_id, base_name, count):
        h, w = image.shape[:2]
        x1, y1, x2, y2 = bbox

        pad_x = int((x2 - x1) * padding_ratio)
        pad_y = int((y2 - y1) * padding_ratio)

        x1 = max(0, int(x1 - pad_x))
        y1 = max(0, int(y1 - pad_y))
        x2 = min(w, int(x2 + pad_x))
        y2 = min(h, int(y2 + pad_y))

        cropped = image[y1:y2, x1:x2]

        new_x, new_y, new_w, new_h = 0.5, 0.5, 1.0, 1.0
        yolo_line = f"{cls_id} {new_x:.6f} {new_y:.6f} {new_w:.6f} {new_h:.6f}"

        new_base = f"{base_name}_aug{count}"
        new_img_path = os.path.join(yolo_img_dir, new_base + ".png")
        new_label_path = os.path.join(yolo_label_dir, new_base + ".txt")
        cv2.imwrite(new_img_path, cropped)
        with open(new_label_path, 'w') as f:
            f.write(yolo_line + "\n")

        class_img_map[cls_id].add(new_base)
        img_class_map[new_base] = {cls_id}

        for i in range(5):
            aug = random.choice(AUGMENTATIONS)
            try:
                aug_data = aug(image=cropped, bboxes=[[0.0, 0.0, 1.0, 1.0]], category_ids=[cls_id])
                aug_img = aug_data['image']
            except:
                continue

            aug_base = f"{base_name}_aug{count}_crop{i}"
            aug_img_path = os.path.join(yolo_img_dir, aug_base + ".png")
            aug_label_path = os.path.join(yolo_label_dir, aug_base + ".txt")
            cv2.imwrite(aug_img_path, aug_img)
            with open(aug_label_path, 'w') as f:
                f.write(yolo_line + "\n")

            class_img_map[cls_id].add(aug_base)
            img_class_map[aug_base] = {cls_id}

    total_success_count = 0

    for cid in sorted_few_class_ids:
        img_list = few_classes[cid]
        print(f"\n📈 클래스 {cid} ({cat_id_to_name.get(cid, cid)}), 목표 증강 수: {max_classes_threshold - len(img_list)}")

        img_cycle = cycle(img_list)
        success_count = 0

        while True:
            current_count = len(class_img_map[cid])
            if current_count >= max_classes_threshold:
                break

            current_counts = {cid_: len(imgs) for cid_, imgs in class_img_map.items()}
            major_classes_set = set(heapq.nlargest(top_n, current_counts, key=current_counts.get))

            base_name = next(img_cycle)
            if "_aug" in base_name:
                continue

            image_path = os.path.join(yolo_img_dir, base_name + ".png")
            label_path = os.path.join(yolo_label_dir, base_name + ".txt")
            if not os.path.exists(image_path) or not os.path.exists(label_path):
                continue

            image = cv2.imread(image_path)
            h, w = image.shape[:2]
            bboxes, class_ids = [], []

            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    cls, x, y, bw, bh = map(float, parts)
                    bboxes.append(yolo_to_xyxy([x, y, bw, bh], w, h))
                    class_ids.append(int(cls))

            # 75개 이상 클래스 포함된 이미지 크롭 증강 적용
            if any(len(class_img_map[c]) >= max_classes_threshold for c in img_class_map[base_name]):
                for box, cls in zip(bboxes, class_ids):
                    if cls == cid:
                        crop_and_save(image, box, cls, base_name, success_count)
                        success_count += 1
                        total_success_count += 1
                        if len(class_img_map[cid]) >= max_classes_threshold:
                            break
                if len(class_img_map[cid]) >= max_classes_threshold:
                    break
            else:
                aug = random.choice(AUGMENTATIONS)
                try:
                    aug_data = aug(image=image, bboxes=bboxes, category_ids=class_ids)
                except:
                    continue

                aug_img = aug_data['image']
                aug_bboxes = aug_data['bboxes']
                aug_class_ids = aug_data['category_ids']

                new_lines = []
                for box, cls in zip(aug_bboxes, aug_class_ids):
                    yolo_box = [max(0, min(1, v)) for v in xyxy_to_yolo(box, w, h)]
                    new_lines.append(f"{cls} {' '.join(f'{v:.6f}' for v in yolo_box)}")

                new_base = f"{base_name}_aug{success_count}"
                new_img_path = os.path.join(yolo_img_dir, new_base + ".png")
                new_label_path = os.path.join(yolo_label_dir, new_base + ".txt")

                cv2.imwrite(new_img_path, aug_img)
                with open(new_label_path, 'w') as f:
                    f.write("\n".join(new_lines))

                for c in set(aug_class_ids):
                    class_img_map[c].add(new_base)
                img_class_map[new_base] = set(aug_class_ids)

                success_count += 1
                total_success_count += 1

    print(f"\n✅ 총 증강된 이미지 수: {total_success_count}개")

In [ ]:
# @title 클래스별 기준 개수보다 많은 이미지 삭제
def fine_tune_delete_by_class_popularity_relaxed(
    yolo_img_dir, yolo_label_dir, target_count=75, margin=10
):
    """
    클래스별 이미지 수가 target_count보다 많을 경우, margin만큼 완화하여
    이미지들을 삭제해 클래스별 이미지 수를 조정합니다.
    - 이미지에 여러 클래스가 포함될 수 있으므로, 삭제 시 모든 클래스의
      이미지 수가 target_count - margin 이상 유지되도록 확인합니다.
    - 삭제 우선순위는 포함된 클래스 개수가 적은 이미지부터 진행합니다.
    - 삭제 시 '_aug' 혹은 '_copy'가 포함된 파일을 우선 랜덤으로 삭제합니다.

    Args:
        yolo_img_dir (str): YOLO 형식 이미지가 저장된 폴더 경로.
        yolo_label_dir (str): YOLO 형식 라벨(.txt) 파일들이 저장된 폴더 경로.
        target_count (int): 클래스별 최소 유지할 이미지 개수 기준.
        margin (int): target_count 대비 완화 허용 범위 (예: target_count - margin 이상 유지).

    Returns:
        None. 삭제 대상 이미지를 실제 파일에서 제거합니다.
    """
    ext = ".png"
    image_to_classes = {}
    class_counts = Counter()

    for fname in os.listdir(yolo_img_dir):
        if not fname.endswith(ext):
            continue
        base = os.path.splitext(fname)[0]
        label_path = os.path.join(yolo_label_dir, base + ".txt")
        classes = set()
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f:
                    cls_id = line.split()[0]
                    classes.add(cls_id)
        image_to_classes[base] = classes
        for cls in classes:
            class_counts[cls] += 1

    current_counts = class_counts.copy()
    class_to_images = defaultdict(set)
    for img, classes in image_to_classes.items():
        for cls in classes:
            class_to_images[cls].add(img)

    to_delete = set()
    classes_sorted = sorted(class_counts.keys(), key=lambda c: class_counts[c], reverse=True)

    print(f"초기 클래스별 이미지 개수: {dict(class_counts)}")

    for cls in classes_sorted:
        if current_counts[cls] <= target_count:
            continue

        imgs_for_cls = list(class_to_images[cls] - to_delete)
        imgs_for_cls.sort(key=lambda img: len(image_to_classes[img]))

        for img in imgs_for_cls:
            img_classes = image_to_classes[img]
            # margin 만큼 완화: 삭제 후 클래스 개수가 target_count - margin 이상이면 OK
            if all(current_counts[c] - 1 >= target_count - margin for c in img_classes):
                to_delete.add(img)
                for c in img_classes:
                    current_counts[c] -= 1
                if current_counts[cls] <= target_count:
                    break

    print(f"\n삭제 예정 이미지 수: {len(to_delete)}")
    print(f"삭제 후 예상 클래스별 이미지 개수: {dict(current_counts)}")

    print("\n삭제 진행 중...")

    def has_special_suffix(base_name):
        return ('_aug' in base_name) or ('_copy' in base_name)

    special_files = [base for base in to_delete if has_special_suffix(base)]
    normal_files = [base for base in to_delete if not has_special_suffix(base)]

    random.shuffle(special_files)
    random.shuffle(normal_files)

    for base in special_files + normal_files:
        img_path = os.path.join(yolo_img_dir, base + ext)
        label_path = os.path.join(yolo_label_dir, base + ".txt")
        if os.path.exists(img_path):
            os.remove(img_path)
            print(f"삭제: {img_path}")
        if os.path.exists(label_path):
            os.remove(label_path)
            print(f"삭제: {label_path}")

    print("\n삭제 완료.")

In [ ]:
# @title yaml파일 생성
def create_yolo_yaml(root_dir, merged_json_path, yaml_path):
    """
    YOLO 학습에 사용할 data.yaml 파일 생성 함수.

    Args:
        root_dir (str): 데이터셋의 루트 경로 (train, val 경로 기준).
        merged_json_path (str): 카테고리 정보를 얻기 위한 JSON 파일 경로.
        yaml_path (str): 생성할 yaml 파일 경로.

    동작:
        - get_master_categories 함수로 카테고리 id와 이름, 인덱스 매핑 정보를 얻음
        - yaml 형식에 맞게 train, val 경로, 클래스 수, 클래스 이름들을 data.yaml 파일로 저장
    """
    # 카테고리 정보 획득: cat_id_to_index (cat_id -> index), cat_id_to_name (cat_id -> 클래스명)
    cat_id_to_index, cat_id_to_name = get_master_categories([merged_json_path])

    # YOLO 클래스 인덱스 → 클래스명 딕셔너리 생성
    names_dict = {
        idx: cat_id_to_name[cat_id]
        for cat_id, idx in cat_id_to_index.items()
    }

    # yaml 파일을 저장할 디렉토리 생성 (존재하지 않으면)
    yaml_dir = os.path.dirname(yaml_path)
    os.makedirs(yaml_dir, exist_ok=True)

    # yaml 파일 생성 및 작성
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write(f"path: {root_dir}\n")                     # 데이터셋 루트 경로
        f.write(f"train: images/train_images\n")           # 학습 이미지 경로 (루트 기준 상대경로)
        f.write(f"val: images/train_images\n")             # 검증 이미지 경로 (루트 기준 상대경로)
        f.write(f"nc: {len(names_dict)}\n")                 # 클래스 수
        f.write("names:\n")                                 # 클래스 이름들
        for idx in range(len(names_dict)):
            class_name = names_dict[idx]
            f.write(f"  {idx}: '{class_name}'\n")

    print(f"data.yaml 파일이 생성되었습니다: {yaml_path}")

## 시각화 메서드

In [ ]:
# @title 바운딩 박스 중복/겹침 확인
def compute_iou(box1, box2):
    """
    두 바운딩 박스(box1, box2)의 IoU(교집합 / 합집합)를 계산

    Args:
        box1, box2: [x1, y1, x2, y2] 형식의 좌표 리스트

    Returns:
        IoU (float): 0.0 ~ 1.0 사이의 겹침 비율
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_w = max(0.0, x2 - x1)
    inter_h = max(0.0, y2 - y1)
    inter_area = inter_w * inter_h
    area1 = max(0.0, (box1[2] - box1[0]) * (box1[3] - box1[1]))
    area2 = max(0.0, (box2[2] - box2[0]) * (box2[3] - box2[1]))
    union_area = area1 + area2 - inter_area
    if union_area == 0:
        return 0.0
    return inter_area / union_area

def xywh_to_xyxy(bbox):
    """
    [x, y, w, h] 형태의 bbox를 [x1, y1, x2, y2] 형태로 변환

    Args:
        bbox: [x, y, w, h]

    Returns:
        [x1, y1, x2, y2]
    """
    x, y, w, h = bbox
    return [x, y, x + w, y + h]

def merge_all_jsons_recursive(json_folder):
    """
    json_folder 내 모든 JSON 파일을 재귀적으로 읽어 COCO 형식의
    이미지, 어노테이션, 카테고리 정보를 모음.

    Returns:
        images_map: {image_id: image_info}
        annotations_map: {image_id: [annotation, ...]}
        categories_map: {category_id: category_info}
        chart_map: {image_id: [chart_label, ...]}  # 이미지별 차트 레이블 목록
    """
    images_map = defaultdict(lambda: None)
    annotations_map = defaultdict(list)
    categories_map = dict()
    chart_map = defaultdict(list)

    for root, _, files in os.walk(json_folder):
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(root, file), 'r') as f:
                    data = json.load(f)

                img_info = data['images'][0]
                img_id = img_info['id']
                images_map[img_id] = img_info

                for ann in data['annotations']:
                    annotations_map[img_id].append(ann)

                for cat in data['categories']:
                    categories_map[cat['id']] = cat

                chart = img_info.get("chart", "")
                chart_map[img_id].append(chart)

    return images_map, annotations_map, categories_map, chart_map

def visualize_overlapping_bboxes_with_all_labels(images_map, annotations_map, categories_map, chart_map, source_img_dir, iou_threshold=0.1):
    """
    이미지별 바운딩 박스 중복(겹침) 여부를 IoU 기준으로 검사하고,
    중복된 바운딩 박스들을 서로 다른 색상으로 시각화하여 출력.

    Args:
        images_map: {image_id: image_info}
        annotations_map: {image_id: [annotation, ...]}
        categories_map: {category_id: category_info}
        chart_map: {image_id: [chart_label, ...]}
        source_img_dir (str): 이미지 파일 경로
        iou_threshold (float): IoU 임계값 (이상인 경우 중복으로 간주)

    Returns:
        None. 각 중복 이미지에 대해 시각화 플롯 출력.
    """
    base_colors = ['red', 'green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'brown']

    for img_id in images_map:
        anns = annotations_map[img_id]
        if len(anns) <= 1:
            continue

        boxes = []
        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])

        overlapping_pairs = []
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                iou = compute_iou(boxes[i], boxes[j])
                if iou >= iou_threshold:
                    overlapping_pairs.append((i, j))

        if not overlapping_pairs:
            continue

        overlapping_indices = set()
        for i, j in overlapping_pairs:
            overlapping_indices.add(i)
            overlapping_indices.add(j)

        img_file = images_map[img_id]['file_name']
        img_path = os.path.join(source_img_dir, img_file)

        # ✅ 파일 열기 실패 시 경고 출력 후 건너뛰기
        try:
            img = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"⚠ 이미지 파일 없음: {img_file}, 건너뜀")
            continue

        print(f"Image: {img_file} - Overlapping bbox pairs (IoU≥{iou_threshold}): {overlapping_pairs}")

        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        ax = plt.gca()

        for i, ann in enumerate(anns):
            x, y, w, h = ann['bbox']
            color = base_colors[i % len(base_colors)] if i in overlapping_indices else 'black'

            rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
            ax.add_patch(rect)

            chart_label = chart_map[img_id][i] if i < len(chart_map[img_id]) else "N/A"
            category_id = ann.get("category_id")
            category_name = categories_map.get(category_id, {}).get("name", "unknown")

            y_text = max(y - 10, 5)
            ax.text(x, y_text, f"[{i}] {category_name} - {chart_label}",
                    color=color, fontsize=12,
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

        plt.axis('off')

        label_lines = []
        for i, ann in enumerate(anns):
            chart_label = chart_map[img_id][i] if i < len(chart_map[img_id]) else "N/A"
            category_id = ann.get("category_id")
            category_name = categories_map.get(category_id, {}).get("name", "unknown")
            label_lines.append(f"[{i}] {category_name} - {chart_label}")

        y_start = 0.95
        for idx, line in enumerate(label_lines):
            plt.figtext(0.01, y_start - idx * 0.03, line, fontsize=10,
                        ha='left', va='top', color='black',
                        bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))

        plt.title(f"{img_file} - Overlapping BBoxes (IoU≥{iou_threshold})")
        plt.show()


In [ ]:
# @title 증강 이미지 바운딩박스 오류 확인
def load_yolo_labels(txt_path, img_w, img_h):
    """
    YOLO 라벨 파일을 읽어 이미지 크기에 맞게 바운딩 박스 좌표를 (cls_id, x1, y1, x2, y2) 형태로 변환하여 반환.

    Args:
        txt_path (str): YOLO 형식 라벨(txt) 파일 경로
        img_w (int): 이미지 너비
        img_h (int): 이미지 높이

    Returns:
        list of tuples: [(cls_id, x1, y1, x2, y2), ...]
    """
    boxes = []
    with open(txt_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls_id, cx, cy, w, h = map(float, parts)
            x1 = (cx - w / 2) * img_w
            y1 = (cy - h / 2) * img_h
            x2 = (cx + w / 2) * img_w
            y2 = (cy + h / 2) * img_h
            boxes.append((int(cls_id), x1, y1, x2, y2))
    return boxes

def is_out_of_bounds(box, img_w, img_h):
    """
    바운딩 박스가 이미지 경계 밖으로 벗어났는지 여부를 검사.

    Args:
        box (tuple): (cls_id, x1, y1, x2, y2)
        img_w (int): 이미지 너비
        img_h (int): 이미지 높이

    Returns:
        bool: 박스가 이미지 경계 밖에 있으면 True, 아니면 False
    """
    _, x1, y1, x2, y2 = box
    return x1 < 0 or y1 < 0 or x2 > img_w or y2 > img_h

def compute_iou_yolo(box1, box2):
    """
    두 바운딩 박스 간 IoU(Intersection over Union) 계산.

    Args:
        box1, box2 (tuple): 각각 (cls_id, x1, y1, x2, y2) 형식의 박스 좌표

    Returns:
        float: 두 박스의 IoU 값 (0~1)
    """
    _, x1, y1, x2, y2 = box1
    _, x3, y3, x4, y4 = box2
    xi1, yi1 = max(x1, x3), max(y1, y3)
    xi2, yi2 = min(x2, x4), min(y2, y4)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

def show_image_with_boxes(img, boxes, out_classes, overlap_classes, fname):
    """
    이미지에 바운딩 박스를 그리고, 경계 밖 박스는 빨간색, 겹치는 박스는 주황색, 정상 박스는 파란색으로 표시하여 시각화.

    Args:
        img (ndarray): RGB 이미지 배열
        boxes (list): [(cls_id, x1, y1, x2, y2), ...] 바운딩 박스 리스트
        out_classes (set): 이미지 밖 박스가 있는 클래스 ID 집합
        overlap_classes (set): 겹치는 박스가 있는 클래스 ID 집합
        fname (str): 이미지 파일명 (플롯 제목에 사용)
    """
    img_h, img_w = img.shape[:2]
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)

    for box in boxes:
        cls_id, x1, y1, x2, y2 = box
        color = 'blue'
        if cls_id in out_classes:
            color = 'red'
        elif cls_id in overlap_classes:
            color = 'orange'

        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                 linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1 - 4, f'{cls_id}', color=color, fontsize=10)

    title = f"{fname}"
    if out_classes or overlap_classes:
        title += "  [⚠️ 확인 필요]"
    ax.set_title(title)
    ax.axis('off')
    plt.show()

def process_image(image_path, label_path, fname):
    """
    단일 이미지와 라벨 파일을 로드하여 바운딩 박스가 이미지 밖에 있는지,
    서로 겹치는 박스가 있는지 검사하고, 이상이 있으면 시각화해서 출력.

    Args:
        image_path (str): 이미지 파일 경로
        label_path (str): YOLO 라벨 파일 경로
        fname (str): 이미지 파일명 (출력용)

    Returns:
        tuple(bool, bool) or None: (이미지 밖 박스 여부, 겹치는 박스 여부)
                                   바운딩 박스가 없으면 None 반환
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"[Error] 이미지 로드 실패: {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_h, img_w = img.shape[:2]

    boxes = load_yolo_labels(label_path, img_w, img_h)
    if not boxes:
        return None

    out_classes = set()
    overlap_classes = set()

    for i, box in enumerate(boxes):
        cls_id = box[0]

        if is_out_of_bounds(box, img_w, img_h):
            out_classes.add(cls_id)

        for j in range(i + 1, len(boxes)):
            iou = compute_iou_yolo(box, boxes[j])
            if iou > 0:
                overlap_classes.add(cls_id)
                overlap_classes.add(boxes[j][0])

    if out_classes or overlap_classes:
        print(f"📂 {fname}")
        if out_classes:
            print(f"  🔴 이미지 밖 클래스: {sorted(out_classes)}")
        if overlap_classes:
            print(f"  🟠 겹치는 클래스: {sorted(overlap_classes)}\n")
        show_image_with_boxes(img, boxes, out_classes, overlap_classes, fname)

    return len(out_classes) > 0, len(overlap_classes) > 0

def process_folder(images_dir, labels_dir):
    """
    지정한 폴더 내 이미지와 라벨들을 모두 검사하여,
    이미지 밖 바운딩 박스 및 겹치는 박스가 있는 이미지 목록과 통계를 출력.

    Args:
        images_dir (str): 이미지 파일들이 있는 디렉토리 경로
        labels_dir (str): YOLO 라벨(txt) 파일들이 있는 디렉토리 경로

    Returns:
        None
    """
    total = 0
    out_count = 0
    overlap_count = 0

    print("🔍 오류 있는 파일 목록:\n")

    for fname in sorted(os.listdir(images_dir)):
        if not fname.endswith('.png'):
            continue

        name_no_ext = os.path.splitext(fname)[0]
        image_path = os.path.join(images_dir, fname)
        label_path = os.path.join(labels_dir, name_no_ext + '.txt')

        if not os.path.exists(label_path):
            continue

        result = process_image(image_path, label_path, fname)
        if result:
            total += 1
            out, overlap = result
            if out:
                out_count += 1
            if overlap:
                overlap_count += 1

    print("📊 통계 요약:")
    print(f"  전체 오류 이미지 수: {total}")
    print(f"  🔴 이미지 밖 박스 포함 수: {out_count}")
    print(f"  🟠 겹치는 박스 포함 수: {overlap_count}")


In [ ]:
# @title 증강 전후 데이터분포 비교
def plot_class_distribution(before_counts, after_counts):
    """
    클래스 분포를 증강 전후로 겹쳐서 비교하는 막대 그래프를 그린다.

    Args:
        before_counts (dict): 증강 전 클래스별 이미지 수
        after_counts (dict): 증강 후 클래스별 이미지 수
    """
    sorted_classes = sorted(before_counts.keys(), key=lambda x: int(x))
    before_values = [before_counts.get(cls, 0) for cls in sorted_classes]
    after_values = [after_counts.get(cls, 0) for cls in sorted_classes]

    bar_width = 0.35
    index = np.arange(len(sorted_classes))

    plt.figure(figsize=(17, 6))
    plt.bar(index, before_values, bar_width, color='#FF6F61', label='Before Augmentation')
    plt.bar(index, after_values, bar_width, color='#6C9ECF', label='After Augmentation', alpha=0.5)

    max_value = max(max(before_values), max(after_values))
    plt.ylim(0, max_value)
    plt.yticks(np.arange(0, max_value + 1, 25))

    for y in np.arange(0, max_value + 1, 25):
        plt.axhline(y=y, color='gray', linestyle='--', alpha=0.5)

    plt.ylabel('이미지수')
    plt.title('Class Distribution Before and After Augmentation')
    plt.xticks(index, sorted_classes, rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# @title 증강 이미지 확인
def read_yolo_labels(label_path):
    """
    YOLO txt 라벨을 읽어 (cls, xywh center normalized) 튜플 리스트 반환.
    """
    boxes = []
    if not os.path.isfile(label_path):
        print(f"Warning: {label_path} does not exist.")
        return boxes
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls_id = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:])
            boxes.append((cls_id, x_c, y_c, w, h))
    return boxes

def generate_class_colors(num_classes):
    """
    클래스별로 랜덤한 RGB 색상 리스트 생성 (고정 시드 사용).
    """
    random.seed(42)
    colors = []
    for _ in range(num_classes):
        colors.append((random.randint(0,255), random.randint(0,255), random.randint(0,255)))
    return colors

def draw_bboxes(img, boxes, class_colors, thickness=4):
    """
    이미지에 각 클래스별 색상으로 바운딩 박스 그리기.
    boxes: (cls_id, x_c, y_c, w, h) 형식 (YOLO normalized)
    """
    h, w = img.shape[:2]
    for cls_id, x_c, y_c, bw, bh in boxes:
        x1 = int((x_c - bw/2) * w)
        y1 = int((y_c - bh/2) * h)
        x2 = int((x_c + bw/2) * w)
        y2 = int((y_c + bh/2) * h)
        color = class_colors[cls_id]
        cv2.rectangle(img, (x1,y1), (x2,y2), color, thickness)

def count_classes_in_boxes(boxes):
    """
    주어진 바운딩 박스 리스트에서 클래스별 개수 세기.
    """
    class_counts = {}
    for cls_id, _, _, _, _ in boxes:
        if cls_id not in class_counts:
            class_counts[cls_id] = 0
        class_counts[cls_id] += 1
    return class_counts

def show_augmented_images_by_class(
    images_dir,
    labels_dir,
    max_aug_images=5,
    num_classes=74
):
    """
    클래스별로 증강된 이미지들을 무작위로 최대 max_aug_images개씩 시각화하여 보여줍니다.

    Args:
        images_dir (str): 증강 이미지들이 저장된 디렉토리 경로
        labels_dir (str): 증강 이미지에 대응하는 YOLO 라벨(txt) 파일들이 있는 디렉토리 경로
        max_aug_images (int): 각 클래스별로 최대 몇 장의 증강 이미지를 보여줄지 지정 (기본 5)
        num_classes (int): 총 클래스 개수 (라벨 색상 및 클래스 인덱스 범위 지정용)
    """
    valid_exts = ('.png')
    all_files = [f for f in os.listdir(images_dir) if f.lower().endswith(valid_exts)]

    # 원본 이미지와 증강 이미지를 구분하기 위해, '_aug' 또는 '_crop'이 포함된 파일만 선택
    aug_files = [f for f in all_files if "_aug" in f or "_crop" in f]
    print(f"Found {len(aug_files)} augmented files: {aug_files}")

    # 클래스별 증강된 이미지를 구분
    class_to_aug_files = {i: [] for i in range(num_classes)}  # 클래스별로 증강 이미지 저장

    for aug_file in aug_files:
        aug_label_path = os.path.join(labels_dir, os.path.splitext(aug_file)[0] + '.txt')
        if not os.path.isfile(aug_label_path):
            continue

        # 라벨 파일에서 클래스 추출
        aug_boxes = read_yolo_labels(aug_label_path)
        for cls_id, _, _, _, _ in aug_boxes:
            if cls_id < num_classes:
                class_to_aug_files[cls_id].append(aug_file)
                break  # 첫 번째 클래스만 가져옴

    class_colors = generate_class_colors(num_classes)

    # 각 클래스별로 증강 이미지 랜덤 출력
    for cls_id, aug_files in class_to_aug_files.items():
        if len(aug_files) > 0:
            # 무작위로 5개의 증강 이미지 선택
            selected_aug_files = random.sample(aug_files, min(max_aug_images, len(aug_files)))

            aug_imgs = []
            for aug_file in selected_aug_files:
                aug_img_path = os.path.join(images_dir, aug_file)
                aug_label_path = os.path.join(labels_dir, os.path.splitext(aug_file)[0] + '.txt')
                aug_img = cv2.imread(aug_img_path)
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
                aug_boxes = read_yolo_labels(aug_label_path)
                draw_bboxes(aug_img, aug_boxes, class_colors)

                aug_imgs.append(aug_img)

            total_imgs = len(aug_imgs)
            plt.figure(figsize=(5 * total_imgs, 6))  # 더 넓고 여유 있는 레이아웃 설정
            plt.suptitle(f"Class {cls_id} - {len(aug_imgs)} Augmented Images", fontsize=16)

            # 증강 이미지들 (가로로 정렬하고, 위쪽에 맞추기)
            for idx, aug_img in enumerate(aug_imgs, start=1):
                plt.subplot(1, total_imgs, idx)  # 1행 여러 열로 배치
                plt.imshow(aug_img)
                plt.title(f"Augmented {idx}")
                plt.axis('off')

            # 여백을 조정하여 이미지 간 간격을 더 넓히고 상단에 맞추기
            plt.subplots_adjust(top=0.85, bottom=0.05, left=0.05, right=0.95, hspace=0.1, wspace=0.2)
            plt.show()

## main

In [ ]:
# 0) 중복/겹치는 바운딩박스 확인
images_map, annotations_map, categories_map, chart_map = merge_all_jsons_recursive(source_label_dir)
visualize_overlapping_bboxes_with_all_labels(images_map, annotations_map, categories_map, chart_map, source_img_dir, iou_threshold=0.1)

In [ ]:
# 1) 어노테이션 병합
merge_coco_jsons(source_label_dir, merged_json_path)

# 2) 카테고리 매핑
master_cat_id_to_idx = get_master_categories([merged_json_path])[0]

# 3) 완전 라벨 데이터 필터링
clean_invalid_yolo_files_from_dir(
    merged_json_path=merged_json_path,
    image_dir=source_img_dir,
    label_dir=source_label_dir
)

# 4) 필터링된 데이터 어노테이션 병합
merge_coco_jsons(source_label_dir, merged_filtered_json_path)

# 4) 이미지 복사
image_files = get_image_files_from_coco_json(merged_filtered_json_path)
copy_images(image_files, source_img_dir, yolo_image_dir)

# 3) YOLO 라벨 변환
convert_coco_to_yolo(
    annotation_file=merged_filtered_json_path,
    source_img_dir=yolo_image_dir,
    target_yolo_dir=yolo_label_dir,
    master_cat_id_to_idx=master_cat_id_to_idx
)

In [ ]:
# 4) train/val 분할
split_train_val(
    image_dir=yolo_image_dir,
    label_dir=yolo_label_dir,
    train_img_dir=os.path.join(yolo_image_dir, "train_images"),
    val_img_dir=os.path.join(yolo_image_dir, "val_images"),
    train_label_dir=os.path.join(yolo_label_dir, "train_images"),
    val_label_dir=os.path.join(yolo_label_dir, "val_images"),
    val_ratio=0.15,
    seed=42
)

# 학습/검증 데이터 이미지/라벨수 확인
print("=== Train Dataset Check ===")
check_images_labels(yolo_train_img_dir, yolo_train_label_dir)

print("\n=== Validation Dataset Check ===")
check_images_labels(yolo_val_img_dir, yolo_val_label_dir)

In [ ]:
# 5) train 이미지 증강. 클래스별 최대 75개
before_counts = count_classes_in_yolo_txt(yolo_train_label_dir)
cat_id_to_index, cat_id_to_name = get_master_categories([merged_json_path])
copy_few_shot_images(
    yolo_img_dir=yolo_train_img_dir,
    yolo_label_dir=yolo_train_label_dir,
    cat_id_to_name=cat_id_to_name,
    max_classes_threshold=76,
    top_n=0
)


In [ ]:
# 6) 75개 이상 존재하는 이미지/라벨 삭제
fine_tune_delete_by_class_popularity_relaxed(
    yolo_train_img_dir,
    yolo_train_label_dir,
    76,
    0
)

In [ ]:
# 7) 증강(삭제) 전후 데이터 분포 확인
after_counts = count_classes_in_yolo_txt(yolo_train_label_dir)

plot_class_distribution(before_counts, after_counts)

In [ ]:
# 8) 증강 데이터 검증
process_folder(
    images_dir=yolo_train_img_dir,
    labels_dir=yolo_train_label_dir
)

In [ ]:
# 9) 증강된 이미지 확인
show_augmented_images_by_class(yolo_train_img_dir, yolo_train_label_dir, max_aug_images=5)

In [ ]:
# 10) data.yaml 파일 생성
cat_id_to_index, cat_id_to_name = get_master_categories([merged_json_path])

# YOLO용 클래스 인덱스 → 클래스명
names_dict = {
    idx: cat_id_to_name[cat_id]
    for cat_id, idx in cat_id_to_index.items()
}

yaml_path = "/kaggle/working/data/data.yaml"
yaml_dir = os.path.dirname(yaml_path)
create_yolo_yaml(root_dir, merged_json_path, yaml_path)

In [ ]:
# 11) 모델학습
model = YOLO("yolov8l.pt")

results = model.train(
    data=yaml_path,
    epochs=150,
    imgsz=640,
    patience=20,
    batch=16,
    device=0,
    lr0=0.01,
    lrf=0.01,
    mosaic=0.5,
    mixup=0.2,
    close_mosaic=10,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    flipud=0.1,
    fliplr=0.1,
    augment=True,
    project="/kaggle/working/data/results",
    name="yolo8l_custom_train_cp_mix",
    exist_ok=True
)

print(results)